In [2]:
import tensorflow as tf
import numpy as np
import psycopg2
import pandas as pd

In [8]:
### load dataset
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database='postgres')

# usage: specify file location, sample size and seed(used by random) 
filepath = '../Data_git_ignore/clean_csv/article_clean.csv' # 250 rows of FakeNewsCorpus
f = 1.0                                            # desired frac
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_sql_query("SELECT * FROM fakenews.article", connection).sample(frac=f, random_state=seed)
#df["content"] = df["content"].astype(str)
# c*reate type_id
#df['type_id'] = df.groupby(['type']).ngroup()

df

,id,domain_id,type_id,url,content_tokenize,content,title,meta_description,scraped_at,updated_at,inserted_at
22476,808451,1,0,http://beforeitsnews.com/forex/2014/08/gbpusd-...,"['gbpusd', 'daily', 'forecast', 'DATE', 'headl...",gbpusd daily forecast DATE headline bitcoin bl...,GBPUSD Daily Forecast: August 22,None,2018-01-25,2018-02-02,2018-02-02
9055,40330,1,0,http://beforeitsnews.com/stories/category/diy....,"['NUM', 'by', 'show', 'tell', 'on', 'tuesday',...",NUM by show tell on tuesday DATE NUM NUM,"Stories in the ""DIY"" category",None,2018-01-25,2018-02-02,2018-02-02
3394,821607,1,0,http://beforeitsnews.com/environment/2010/11/f...,"['following', 'the', 'money', 'of', 'readers',...",following the money of readers think this stor...,Following the money…,None,2018-01-25,2018-02-02,2018-02-02
5552,23931,1,0,http://beforeitsnews.com/libertarian/2013/12/t...,"['tampon', 'earrings', 'parody', 'of', 'reader...",tampon earrings parody of readers think this s...,Tampon Earrings Parody,None,2018-01-25,2018-02-02,2018-02-02
80575,407069,117,3,http://www.abovetopsecret.com/forum/thread1171...,"['edit', 'on', 'DATE', 'by', 'tonycodes', 'bec...",edit on DATE by tonycodes because no reason gi...,"Help Mexico's Children!, page 1",None,2018-01-25,2018-02-02,2018-02-02
...,...,...,...,...,...,...,...,...,...,...,...
73349,317563,117,3,http://www.abovetopsecret.com/forum/thread7758...,"['before', 'some', 'people', 'decide', 'to', '...",before some people decide to crap on this thre...,"Charleston County, SC and San Diego, CA., page 1",None,2018-01-25,2018-02-02,2018-02-02
109259,913747,210,6,http://occupydemocrats.com/2016/09/16/trump-ju...,"['mashshare', 'the', 'media', 'is', 'full', 'o...",mashshare the media is full of selfrighteous i...,Trump Just Tricked Gullible Media Into Promoti...,None,2018-01-25,2018-02-02,2018-02-02
50057,407219,59,2,http://investmentwatchblog.com/almost-all-deve...,"['by', 'mark', 'angelides', 'the', 'idea', 'th...",by mark angelides the idea that to ask voters...,Almost ALL Developed Nations Require Voter ID ...,None,2018-01-25,2018-02-02,2018-02-02
5192,777382,1,0,http://beforeitsnews.com/politics/2016/05/did-...,"['did', 'the', 'arabs', 'betray', 'palestine',...",did the arabs betray palestine headline bitcoi...,Did the Arabs betray Palestine?,None,2018-01-25,2018-02-02,2018-02-02


In [25]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# v = TfidfVectorizer()

# #df['content'] = df['content'].apply(' '.join)

# x = v.fit_transform(df["content"])

In [24]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(x.toarray(), df["type_id"], test_size=0.2, random_state=0)

In [58]:
df

,type_id,content
22476,0,gbpusd daily forecast DATE headline bitcoin bl...
9055,0,NUM by show tell on tuesday DATE NUM NUM
3394,0,following the money of readers think this stor...
5552,0,tampon earrings parody of readers think this s...
80575,3,edit on DATE by tonycodes because no reason gi...
...,...,...
73349,3,before some people decide to crap on this thre...
109259,6,mashshare the media is full of selfrighteous i...
50057,2,by mark angelides the idea that to ask voters...
5192,0,did the arabs betray palestine headline bitcoi...


In [61]:
### SELECT TRAIN, TEST, VALIDATE ###

# Select columns for dataset for furthere split
X_columns = ['content'] # <- n number of columns
y_column  = ['type_id'] # <- max one column

# Select columns for dataframe
df = df[y_columns+X_columns]


# This can generate a dataset with random purmutation and a max size for each type(can be smaller if desired max is not possible)

# max size for type
max_size = 25000
# traning_set ratio - splits data into traning=ratio,  test and validate=(1-ratio)/2 ex. train=80%, test=10%, validate=10%
ratio=0.7


# Labels to include - ['fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable'] - all labels

# use_types = ['fake', 'satire', 'bias', 'conspiracy', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable'] # <- string labes
use_types = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]                                                                                    # <- int labels

# Random seed
rnd = 1

# initialize dataframes
train    = pd.DataFrame(columns = df.columns)
test     = pd.DataFrame(columns = df.columns)
validate = pd.DataFrame(columns = df.columns)

# add type to test splits
for t in use_types:

    # type size
    type_size = df[y_columns[0]].loc[df[y_columns[0]] == t].value_counts().min()

    if type_size > 0:
        # set size of type slice
        if type_size < max_size:
            tmp = df.loc[df[y_columns[0]] == t].sample(n = type_size, random_state=rnd)
        else:
            tmp = df.loc[df[y_columns[0]] == t].sample(n = max_size, random_state=rnd)

        # split current type
        train_tmp, test_tmp, validate_tmp = np.split(tmp, [int(ratio * len(tmp)), int(((1-ratio)/2 + ratio) * len(tmp))])

        # add tmp to dataframes
        train    = pd.concat([train, train_tmp])
        test     = pd.concat([test, test_tmp])
        validate = pd.concat([validate, validate_tmp])
    
        # print split shape
        print("=>", t, tmp.shape, train_tmp.shape, validate_tmp.shape, validate_tmp.shape)
    else:
        print("\n[Error]", t, "- not in dataset from [y_column] <-", y_column)

print("\n[Final split]\ntrain, test, validate ==>", train.shape, test.shape, validate.shape)

# Create X, y split for all sets
X_train    = train[X_columns]
y_train    = train[y_column]
X_test     = test[X_columns]
y_test     = test[y_column]
X_validate = validate[X_columns]
y_validate = validate[y_column]

# Free memory
del train
del test
del validate

=> 0 (25000, 2) (17500, 2) (3750, 2) (3750, 2)
=> 1 (7513, 2) (5259, 2) (1127, 2) (1127, 2)
=> 2 (25000, 2) (17500, 2) (3750, 2) (3750, 2)
=> 3 (25000, 2) (17500, 2) (3750, 2) (3750, 2)
=> 4 (11468, 2) (8027, 2) (1721, 2) (1721, 2)
=> 5 (3112, 2) (2178, 2) (467, 2) (467, 2)
=> 6 (19334, 2) (13533, 2) (2901, 2) (2901, 2)
=> 7 (25000, 2) (17500, 2) (3750, 2) (3750, 2)
=> 8 (25000, 2) (17500, 2) (3750, 2) (3750, 2)
=> 9 (5222, 2) (3655, 2) (784, 2) (784, 2)

[Final split]
train, test, validate ==> (120152, 2) (25747, 2) (25750, 2)


In [21]:
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train.values, epochs=3)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [17]:
# dette for for at trække om modellen er overfittet. val_loss og val_acc skal minde meget om det man får i fittet. 
#Det er ok at loss og acc er lidt højre på testen. De må ikke være fortæt eller for længt væk fra hinaden
val_loss, val_acc = model.evaluate(X_test,y_test.values)
print(val_loss, val_acc)

2400/2400 [==============================] - 5s 2ms/sample - loss: 0.0928 - accuracy: 0.9675
0.09277646052340667 0.9675
